<a href="https://colab.research.google.com/github/bogdanbabych/experiments_NLTK/blob/main/fst_script_71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Converting the input/output format -- FST

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/7101da9baf2b4197b115/?dl=1

In [ ]:
!mv index.html?dl=1 TbiLLC-data.tgz
!tar xvzf TbiLLC-data.tgz

In [3]:
!cp /content/TbiLLC-data/FST.txt /content/FST.txt

In [4]:
%%bash
awk ' NR>=3&&NR<=23 ' /content/TbiLLC-data/Input_Output.txt >testIn.txt
awk ' NR>=28&&NR<=44 ' /content/TbiLLC-data/Input_Output.txt >testOutCurrent.txt
awk ' NR>=51&&NR<=74 ' /content/TbiLLC-data/Input_Output.txt >testOutDesired.txt

In [15]:
%%bash
awk ' NR>=1&&NR<=5 ' FST.txt >FST101all.txt
awk ' NR>=6&&NR<=8 ' FST.txt >FST102brave.txt
awk ' NR>=9&&NR<=10 ' FST.txt >FST103warrior.txt
awk ' NR>=11&&NR<=12 ' FST.txt >FST104always.txt
awk ' NR>=13&&NR<=15 ' FST.txt >FST105very.txt
awk ' NR>=16&&NR<=18 ' FST.txt >FST106selfsacrifice.txt
awk ' NR>=19&&NR<=20 ' FST.txt >FST107protect.txt
awk ' NR>=21&&NR<=25 ' FST.txt >FST108all.txt
awk ' NR>=26&&NR<=27 ' FST.txt >FST109weak.txt
awk ' NR>=28&&NR<=29 ' FST.txt >FST110person.txt
awk ' NR>=30&&NR<=31 ' FST.txt >FST100eoffstop.txt

In [16]:
!cat FST101all.txt FST100eoffstop.txt >FST201all.txt
!cat FST102brave.txt FST100eoffstop.txt >FST202brave.txt
!cat FST103warrior.txt FST100eoffstop.txt >FST203warrior.txt
!cat FST104always.txt FST100eoffstop.txt >FST204always.txt
!cat FST105very.txt FST100eoffstop.txt >FST205very.txt
!cat FST106selfsacrifice.txt FST100eoffstop.txt >FST206selfsacrifice.txt
!cat FST107protect.txt FST100eoffstop.txt >FST207protect.txt
!cat FST108all.txt FST100eoffstop.txt >FST208all.txt
!cat FST109weak.txt FST100eoffstop.txt >FST209weak.txt
!cat FST110person.txt FST100eoffstop.txt >FST210person.txt


In [22]:
!diff FST201all.txt FST208all.txt

1c1
< ​ყველა
---
> ყველა


In [8]:
import sys, os


In [20]:
#CHAT optimized SCRIPT

import re

def convert_string(input_string: str) -> str:
    """Convert a string from TIGER format to NLTK format."""
    # Extract the values for case, pos, and SF from the input string
    match = re.match(r"\[case=([^ ]*?) pos=([^ ]*?)\]", input_string)
    if match:
        case = match.group(1)
        pos = match.group(2)

        # only keep the last part of `case`,
        # e.g. `*.Stm` -> `Stm` and `Sg*.Inst` -> `Inst`
        case = case.split('.')[-1]



        # I'm making up grammar rules here
        sf = "HEAD" if pos == "NN" else "MD"

        # Build the output string based on the values extracted from the input string
        if pos == "ADJ":
            output_string = f"Adj1[SF={sf}, case={case}]"
        elif pos == "PRN_DT":
            output_string = f"PRN_DT[SF={sf}, case={case}]"
        elif pos == "NN":
            output_string = f"NN[SF={sf}, case={case}]"
        elif pos == "TADV":
            output_string = f"Adv1[SF={sf}]"
        elif pos == "IADV":
            output_string = f"Adv2[SF={sf}, case={case}]"
        elif pos == "$.":
            output_string = f"$.[case={case}]"
        else:
            output_string = "Invalid input string"

    else:
        output_string = input_string

    return output_string

def parse_ffst_file(file: str, fileout = 'out70.txt'):
    fout = open(fileout, 'w')
    with open(file) as f:
        lines = [x.strip() for x in f.readlines()]

    sections = []
    current_section = []
    for line in lines:
        if re.match(r"IV01\[", line):  # line contains verb grammatical description
            current_section.append(line)
            agr = 'IV01'
        elif re.match(r"\[case", line):  # line contains grammatical description
            current_section.append(line)
        else:  # line contains a word
            if current_section:  # current section is not empty, i.e. this word starts a new section
                sections.append(current_section)
                current_section = [line]
            else:  # first word of the file
                current_section.append(line)

    # Adding SF=HEAD in Grammatical description for non Noun input/converted strings
    for section in sections:
        word = section[0]
        for gram_desc in section[1:]:
            gram_desc = convert_string(gram_desc)
            match = re.match(r'^(.*\[SF=)(HEAD|MD)(.*\])$', gram_desc)
            if match:
                prefix = match.group(1)
                sf_value = match.group(2)
                suffix = match.group(3)
                if sf_value == "HEAD":
                    print(gram_desc, "->", "'" + word + "'", file=fout)  # SF=HEAD, print as is
                    print('', file=fout)
                elif sf_value == "MD":
                    print(gram_desc, "->", "'" + word + "'", file=fout)  # SF=MD, print original string
                    new_desc = f"{prefix}HEAD{suffix}"  # Substitute SF=MD with SF=HEAD
                    print(new_desc, "->", "'" + word + "'", file=fout)  # Print modified string
                    print('', file=fout)
            else:
                print("Invalid string format:", gram_desc, file=fout)
    fout.flush()
    fout.close()


In [10]:
parse_ffst_file('testIn.txt', fileout = 'testOutDevelopment.txt')

In [11]:
parse_ffst_file('FST.txt', fileout = 'out70.txt')

In [21]:
parse_ffst_file('FST201all.txt', fileout = 'FST201all.out70.txt')
parse_ffst_file('FST202brave.txt', fileout = 'FST202brave.out70.txt')
parse_ffst_file('FST203warrior.txt', fileout = 'FST203warrior.out70.txt')
parse_ffst_file('FST204always.txt', fileout = 'FST204always.out70.txt')
parse_ffst_file('FST205very.txt', fileout = 'FST205very.out70.txt')
parse_ffst_file('FST206selfsacrifice.txt', fileout = 'FST206selfsacrifice.out70.txt')
parse_ffst_file('FST207protect.txt', fileout = 'FST207protect.out70.txt')
parse_ffst_file('FST208all.txt', fileout = 'FST208all.out70.txt')
parse_ffst_file('FST209weak.txt', fileout = 'FST209weak.out70.txt')
parse_ffst_file('FST210person.txt', fileout = 'FST210person.out70.txt')

In [18]:
rm FST1*.txt

In [14]:
rm FST2*.txt

In [23]:
!diff FST208all.out70.txt FST201all.out70.txt

1,2c1,2
< Adj1[SF=MD, case=Stm] -> 'ყველა'
< Adj1[SF=HEAD, case=Stm] -> 'ყველა'
---
> Adj1[SF=MD, case=Stm] -> '​ყველა'
> Adj1[SF=HEAD, case=Stm] -> '​ყველა'
4,5c4,5
< Adj1[SF=MD, case=Nom] -> 'ყველა'
< Adj1[SF=HEAD, case=Nom] -> 'ყველა'
---
> Adj1[SF=MD, case=Nom] -> '​ყველა'
> Adj1[SF=HEAD, case=Nom] -> '​ყველა'
7,8c7,8
< Adj1[SF=MD, case=Voc] -> 'ყველა'
< Adj1[SF=HEAD, case=Voc] -> 'ყველა'
---
> Adj1[SF=MD, case=Voc] -> '​ყველა'
> Adj1[SF=HEAD, case=Voc] -> '​ყველა'
10,11c10,11
< PRN_DT[SF=MD, case=Nom] -> 'ყველა'
< PRN_DT[SF=HEAD, case=Nom] -> 'ყველა'
---
> PRN_DT[SF=MD, case=Nom] -> '​ყველა'
> PRN_DT[SF=HEAD, case=Nom] -> '​ყველა'
